<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/03_readmission_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!rm -rf mimic-readmission-analysis
!git clone https://github.com/seoyeon0905/mimic-readmission-analysis.git
%cd mimic-readmission-analysis

/content
Cloning into 'mimic-readmission-analysis'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 172 (delta 10), reused 5 (delta 5), pack-reused 155 (from 1)
Receiving objects: 100% (172/172), 385.81 KiB | 8.97 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/mimic-readmission-analysis


In [3]:
%cd notebooks
!ls

/content/mimic-readmission-analysis/notebooks
01_data_loading.ipynb  02_cohort_definition.ipynb
